# Homework 2, Problem 5

In this problem we look at weather and how it impacts trading on the New York stock enchange. Complete this notebook and keep it in a homework 2 repo. Submit the repo link though the blackboard.

**You are free to add implmentation or markdown cells to make your notebook clearer!!**

## Data:

The following two datasets are our focus

* Weather data [NOAA-GHCN](https://registry.opendata.aws/noaa-ghcn/)
* Stock Exchange Data [Yahoo Finance](https://finance.yahoo.com/quote/%5ENYA/history?ltr=1) 



## Part 1: Download The Weather Data




Download a year of weather data.

The Raw GHCN files don't have column headers, so we manually add them in. It's safer to at this point read in everything as an object & then parse to the correct type once you extract the variables you're interested in. 
This information can be found in https://docs.opendata.aws/noaa-ghcn-pds/readme.html

In [2]:
import urllib 
import pandas as pd
import numpy as np
from dask.base import compute
import dask.dataframe as dd
import dask.bag as db
import dask.diagnostics as dg

We're using Dask for the lazy evaluation properties (it will only try to run the computations at the end, hopefully after the data has been filtered down) because the dataset is very large. We set the storage options to `anon=True` because this data is public. Otherwise this kwarg is where we'd pass in the AWS authorization keys. 

In [3]:
# Let's load in the data for 1992
YEAR = 1992

names = ['ID', 'DATE', 'ELEMENT', 'DATA_VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG', 'OBS-TIME']
ds = dd.read_csv(f's3://noaa-ghcn-pds/csv/{YEAR}.csv', storage_options={'anon':True},  names=names, memory_map=False, 
                  dtype={'DATA_VALUE':'object'}, parse_dates=['DATE', 'OBS-TIME'])

In [4]:
# You can check the data
print(ds.columns)
print(ds.dtypes)

Index(['ID', 'DATE', 'ELEMENT', 'DATA_VALUE', 'M-FLAG', 'Q-FLAG', 'S-FLAG',
       'OBS-TIME'],
      dtype='object')
ID                    object
DATE          datetime64[ns]
ELEMENT               object
DATA_VALUE            object
M-FLAG                object
Q-FLAG                object
S-FLAG                object
OBS-TIME              object
dtype: object


In [5]:
# Print out the first few rows
ds.head()

,ID,DATE,ELEMENT,DATA_VALUE,M-FLAG,Q-FLAG,S-FLAG,OBS-TIME
0,CA002303986,1992-01-01,TMAX,-70,NaN,NaN,C,nan
1,CA002303986,1992-01-01,TMIN,-240,NaN,NaN,C,nan
2,CA002303986,1992-01-01,PRCP,4,NaN,NaN,C,nan
3,CA002303986,1992-01-01,SNOW,4,NaN,NaN,C,nan
4,CA002303986,1992-01-01,SNWD,420,NaN,NaN,C,nan


Now we want to parse out the station ID list. We are using [pandas.read_fwf](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.read_fwf.html#pandas.read_fwf) because this file is a fixed format width table rather than a csv file. 
We explicitly pass in the extents of the fixed width field because Pandas has trouble inferring what belongs in the `STATE` column versus in the `NAME` column. We obtained these extents from the readme https://docs.opendata.aws/noaa-ghcn-pds/readme.html

In [7]:
# {column name:extents of the fixed-width fields}
columns = {"ID": (0,11), "LATITUDE": (12, 20), "LONGITUDE": (21, 30), "ELEVATION": (31, 37),"STATE": (38, 40),
           "NAME": (41, 71), "GSN FLAG": (72, 75), "HCN/CRN FLAG": (76, 79),"WMO ID": (80, 85)}

In [8]:
df = pd.read_fwf("http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-stations.txt", 
                    colspecs=list(columns.values()), names=list(columns.keys()))

In [9]:
df.head()

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN FLAG,HCN/CRN FLAG,WMO ID
0,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
1,ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN,NaN
2,AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,NaN,41196.0
3,AEM00041194,25.2550,55.3640,10.4,NaN,DUBAI INTL,NaN,NaN,41194.0
4,AEM00041217,24.4330,54.6510,26.8,NaN,ABU DHABI INTL,NaN,NaN,41217.0


In [10]:
# You should be looking for those in the New York area like Central Park, JFK, LGA and Newark airport.
NYNJ = df[df['STATE'].isin(['NY', 'NJ'])]
NYNJ.head()

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN FLAG,HCN/CRN FLAG,WMO ID
73674,US1NJAT0001,39.5483,-74.8671,31.4,NJ,BUENA VISTA TWP 2.6 NNE,NaN,NaN,NaN
73675,US1NJAT0002,39.5565,-74.8048,14.0,NJ,FOLSOM 3.2 SE,NaN,NaN,NaN
73676,US1NJAT0003,39.4747,-74.7107,5.5,NJ,HAMILTON TWP 2.1 SE,NaN,NaN,NaN
73677,US1NJAT0005,39.6404,-74.8261,29.9,NJ,HAMMONTON 3.3 WSW,NaN,NaN,NaN
73678,US1NJAT0009,39.3346,-74.5759,5.8,NJ,LINWOOD 0.7 SSW,NaN,NaN,NaN


Central Park is coded in shorthand, so we used the NOAA web portal to look up the correct ID
https://www.ncdc.noaa.gov/cdo-web/datasets/GHCND/stations/GHCND:USW00094728/detail

In [11]:
NYNJ[NYNJ['ID'].str.contains('USW00094728')]

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN FLAG,HCN/CRN FLAG,WMO ID
114226,USW00094728,40.7789,-73.9692,39.6,NY,NEW YORK CNTRL PK TWR,NaN,HCN,72506.0


In [12]:
# Airports + Central Park
apcp = NYNJ[NYNJ['NAME'].str.endswith('AP') | NYNJ['ID'].str.contains('USW00094728')]
apcp.head()

,ID,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,GSN FLAG,HCN/CRN FLAG,WMO ID
100219,USC00305840,43.1139,-78.9353,179.2,NY,NIAGARA FALLS INTL AP,NaN,NaN,NaN
112764,USW00004724,43.1072,-78.9453,178.3,NY,NIAGARA FALLS INTL AP,NaN,NaN,NaN
112769,USW00004742,44.6500,-73.4667,71.9,NY,PLATTSBURGH INTL AP,NaN,NaN,NaN
112775,USW00004781,40.7939,-73.1017,25.6,NY,ISLIP LI MACARTHUR AP,NaN,NaN,72505.0
112779,USW00004789,41.5092,-74.2650,111.3,NY,MONTGOMERY ORANGE AP,NaN,NaN,NaN


What we're interested in is the IDs, which we will use for our dataset to obtain only the stations of interest. We are going to join our two dataframes on the ID column so that we have all the information in every row.  We are removing the flags since they have neither computational nor necessary identification information. 

we do not use `.compute()` to resolve the computation because it's better to hold off until the completetion of feature selection and engineering described below. If you'd like a fully computed dataframe, the code is 
```python


In [13]:
nyds = ds.merge(apcp[['ID', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'STATE', 'NAME']], on='ID').compute()


In [14]:
nyds.head()

,ID,DATE,ELEMENT,DATA_VALUE,M-FLAG,Q-FLAG,S-FLAG,OBS-TIME,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME
0,USW00094790,1992-01-01,TMAX,61,NaN,NaN,0,2400,43.9922,-76.0217,96.9,NY,WATERTOWN INTL AP
1,USW00094790,1992-01-01,TMIN,-133,NaN,NaN,0,2400,43.9922,-76.0217,96.9,NY,WATERTOWN INTL AP
2,USW00094790,1992-01-01,PRCP,0,NaN,NaN,0,2400,43.9922,-76.0217,96.9,NY,WATERTOWN INTL AP
3,USW00094790,1992-01-01,SNOW,0,NaN,NaN,0,nan,43.9922,-76.0217,96.9,NY,WATERTOWN INTL AP
4,USW00094790,1992-01-01,SNWD,0,NaN,NaN,0,nan,43.9922,-76.0217,96.9,NY,WATERTOWN INTL AP


## Part 2: Downoad Stock Price Data

Here the idea is to get the finance data from Yahoo finace.  It's already the right date range in general:

In [15]:
finance_df = pd.read_csv("https://query1.finance.yahoo.com/v7/finance/download/%5ENYA?period1=694224000&period2=725760000&interval=1d&events=history")
finance_df = finance_df.rename(columns={"Date": "DATE"})

In [16]:
finance_df.head()

,DATE,Open,High,Low,Close,Adj Close,Volume
0,1992-01-02,2423.179932,2423.179932,2423.179932,2423.179932,2423.179932,0
1,1992-01-03,2435.659912,2435.659912,2435.659912,2435.659912,2435.659912,0
2,1992-01-06,2430.370117,2430.370117,2430.370117,2430.370117,2430.370117,0
3,1992-01-07,2428.679932,2428.679932,2428.679932,2428.679932,2428.679932,0
4,1992-01-08,2434.389893,2434.389893,2434.389893,2434.389893,2434.389893,0


You can do an inner join for the dates from the financial dataset and the new york weather dataset, to get all your features ready, please do that here:

In [17]:
# your join on dates goes here:
#ny_df = nyds.compute()
ny_df = nyds
ny_df["DATE"] = pd.to_datetime(ny_df["DATE"])
finance_df["DATE"] = pd.to_datetime(finance_df["DATE"])

In [18]:
final_df = ny_df.merge(finance_df, on='DATE')
final_df.head()

,ID,DATE,ELEMENT,DATA_VALUE,M-FLAG,Q-FLAG,S-FLAG,OBS-TIME,LATITUDE,LONGITUDE,ELEVATION,STATE,NAME,Open,High,Low,Close,Adj Close,Volume
0,USW00094790,1992-01-02,TMAX,22,NaN,NaN,0,2400,43.9922,-76.0217,96.9,NY,WATERTOWN INTL AP,2423.179932,2423.179932,2423.179932,2423.179932,2423.179932,0
1,USW00094790,1992-01-02,TMIN,-94,NaN,NaN,0,2400,43.9922,-76.0217,96.9,NY,WATERTOWN INTL AP,2423.179932,2423.179932,2423.179932,2423.179932,2423.179932,0
2,USW00094790,1992-01-02,PRCP,0,NaN,NaN,0,2400,43.9922,-76.0217,96.9,NY,WATERTOWN INTL AP,2423.179932,2423.179932,2423.179932,2423.179932,2423.179932,0
3,USW00094790,1992-01-02,SNOW,0,NaN,NaN,0,nan,43.9922,-76.0217,96.9,NY,WATERTOWN INTL AP,2423.179932,2423.179932,2423.179932,2423.179932,2423.179932,0
4,USW00094790,1992-01-02,SNWD,0,NaN,NaN,0,nan,43.9922,-76.0217,96.9,NY,WATERTOWN INTL AP,2423.179932,2423.179932,2423.179932,2423.179932,2423.179932,0


In [19]:
final_df = final_df.drop(['M-FLAG','Q-FLAG','S-FLAG','OBS-TIME'],axis=1)

In [20]:
final_df['DATA_VALUE'] = pd.to_numeric(final_df['DATA_VALUE'])
final_df['ELEMENT'].unique()

array(['TMAX', 'TMIN', 'PRCP', 'SNOW', 'SNWD', 'TOBS', 'WT01', 'FMTM',
       'PGTM', 'TSUN', 'WDF1', 'WDFG', 'WSF1', 'WSFG', 'ACMH', 'ACSH',
       'AWND', 'WT08', 'WT16', 'WT14', 'WT06', 'WT15', 'WT18', 'WT04',
       'WT17', 'WT02', 'WT03', 'WT05', 'WT09', 'WESD', 'WT22', 'WT11'],
      dtype=object)

In [21]:
final_df.head()
nyc_weather_df = final_df[(final_df['ELEMENT'] == 'PRCP') | (final_df['ELEMENT'] == 'SNOW') | (final_df['ELEMENT'] == 'SNWD') 
                    | (final_df['ELEMENT'] == 'WT09') | (final_df['ELEMENT'] == 'WT07') |(final_df['ELEMENT'] == 'WT11')
                    | (final_df['ELEMENT'] == 'WT14') |(final_df['ELEMENT'] == 'WT16')|(final_df['ELEMENT'] == 'WT17')
                    | (final_df['ELEMENT'] == 'WT04')|(final_df['ELEMENT'] == 'WT05') | (final_df['ELEMENT'] == 'WT18')]
nyc_weather_df = pd.pivot_table(nyc_weather_df, values='DATA_VALUE', index=['DATE'], columns=['ELEMENT'],fill_value=0)

In [22]:
nyc_weather_df.head(5)

ELEMENT,PRCP,SNOW,SNWD,WT04,WT05,WT09,WT11,WT14,WT16,WT17,WT18
DATE,,,,,,,,,,,
1992-01-02,0.000000,0.000000,8.941176,0,0,0,0,1,1,0,0
1992-01-03,0.294118,0.000000,7.470588,0,0,0,0,1,1,0,0
1992-01-06,0.294118,0.000000,0.000000,0,0,0,0,1,1,0,1
1992-01-07,2.294118,2.294118,0.000000,1,0,0,0,1,1,0,1
1992-01-08,0.176471,0.176471,0.000000,0,0,0,0,0,0,0,1


In [23]:
nyc_weather_df["SNOW"] = nyc_weather_df["SNOW"].astype('int')
nyc_weather_df["PRCP"] = nyc_weather_df["PRCP"].astype('int')

In [24]:
from sklearn import preprocessing
cols = ['Open','High','Low','Close','Adj Close']
x = finance_df[['Open','High','Low','Close','Adj Close']].values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
finance_df_norm = pd.DataFrame(x_scaled, columns=cols)
finance_df_norm['DATE'] = finance_df["DATE"]
finance_df_norm.head()

,Open,High,Low,Close,Adj Close,DATE
0,0.465631,0.465631,0.465631,0.465631,0.465631,1992-01-02
1,0.514483,0.514483,0.514483,0.514483,0.514483,1992-01-03
2,0.493777,0.493777,0.493777,0.493777,0.493777,1992-01-06
3,0.487160,0.487160,0.487160,0.487160,0.487160,1992-01-07
4,0.509512,0.509512,0.509512,0.509512,0.509512,1992-01-08


##### Merge fincial data and stock weather data

In [23]:
stock_weather_df = pd.merge(finance_df_norm, nyc_weather_df, on='DATE')
stock_weather_df.head()

,Open,High,Low,Close,Adj Close,DATE,PRCP,SNOW,SNWD,WT04,WT05,WT09,WT11,WT14,WT16,WT17,WT18
0,0.465631,0.465631,0.465631,0.465631,0.465631,1992-01-02,0,0,8.941176,0,0,0,0,1,1,0,0
1,0.514483,0.514483,0.514483,0.514483,0.514483,1992-01-03,0,0,7.470588,0,0,0,0,1,1,0,0
2,0.493777,0.493777,0.493777,0.493777,0.493777,1992-01-06,0,0,0.000000,0,0,0,0,1,1,0,1
3,0.487160,0.487160,0.487160,0.487160,0.487160,1992-01-07,2,2,0.000000,1,0,0,0,1,1,0,1
4,0.509512,0.509512,0.509512,0.509512,0.509512,1992-01-08,0,0,0.000000,0,0,0,0,0,0,0,1


In [25]:
nyc_weather_df["PRCP"] = np.array(nyc_weather_df["PRCP"])
nyc_weather_df["PRCP"] = np.where(nyc_weather_df["PRCP"]>0,1,0)
nyc_weather_df["SNOW"] = np.array(nyc_weather_df["SNOW"])
nyc_weather_df["SNOW"] = np.where(nyc_weather_df["SNOW"]>0,1,0)
nyc_weather_df["PRCP"] = pd.Series(nyc_weather_df["PRCP"])
nyc_weather_df["SNOW"] = pd.Series(nyc_weather_df["SNOW"])
nyc_weather_df.head()

ELEMENT,PRCP,SNOW,SNWD,WT04,WT05,WT09,WT11,WT14,WT16,WT17,WT18
DATE,,,,,,,,,,,
1992-01-02,0,0,8.941176,0,0,0,0,1,1,0,0
1992-01-03,0,0,7.470588,0,0,0,0,1,1,0,0
1992-01-06,0,0,0.000000,0,0,0,0,1,1,0,1
1992-01-07,1,1,0.000000,1,0,0,0,1,1,0,1
1992-01-08,0,0,0.000000,0,0,0,0,0,0,0,1


## Part 3: Creating and Selecting Variables

Pull out and encode the various variables listed below and set up these varaibles at least initially in a pandas data frame.

### Weather variables

* raining:
    - 0 - wasn't raining
    - 1 - was raining
* rain intensity:
    - 0 -low
    - 1 - medium
    - 2 - high
* rain duration in hours
* snowing:
    - 0 - wasn't snowing
    - 1 - was snowing
* snow intensity:
    - 0 - low
    1 - medium
    2 - high
* snow duration in hours
* windy:
    - 0 - low
    - 1 - medium
    - 2 - high

### Market Variables 

* Market Open
* Market Close
* Market High
* Market Low
* Market Volume


Make sure you have aligned the data by date in a pandas data frame. Show the counts and the summary stats.

In [26]:
nyc_weather_df['Rain_intensity'] = pd.Series(0, index=nyc_weather_df.index)
nyc_weather_df['Snow_intensity'] = pd.Series(0, index=nyc_weather_df.index)
nyc_weather_df['Wind_intensity'] = pd.Series(0, index=nyc_weather_df.index)

In [27]:
nyc_weather_df["Rain_intensity"] = nyc_weather_df.apply(lambda row : 2 if row['WT17'] == 1 else (1 if row['WT16'] == 1 else 0), axis=1)
nyc_weather_df["Snow_intensity"] = nyc_weather_df.apply(lambda row : 2 if row['WT18'] == 1 else (1 if row['WT05'] == 1 else 0), axis=1)
nyc_weather_df["Wind_intensity"] = nyc_weather_df.apply(lambda row : 0 if row['WT09'] == 1 else (2 if row['WT11'] == 1 else 1), axis=1)

In [28]:
nyc_weather_df = nyc_weather_df.drop(['SNWD', 'WT04', 'WT05', 'WT09', 'WT11', 'WT14', 'WT16',
       'WT17', 'WT18'], axis=1)

In [28]:
#finance_df.head()

In [29]:
joined_data = nyc_weather_df.merge(finance_df_norm, on='DATE')
joined_data.head()

,DATE,PRCP,SNOW,Rain_intensity,Snow_intensity,Wind_intensity,Open,High,Low,Close,Adj Close
0,1992-01-02,0,0,1,0,1,0.465631,0.465631,0.465631,0.465631,0.465631
1,1992-01-03,0,0,1,0,1,0.514483,0.514483,0.514483,0.514483,0.514483
2,1992-01-06,0,0,1,2,1,0.493777,0.493777,0.493777,0.493777,0.493777
3,1992-01-07,1,1,1,2,1,0.487160,0.487160,0.487160,0.487160,0.487160
4,1992-01-08,0,0,0,2,1,0.509512,0.509512,0.509512,0.509512,0.509512


## Part 4: Feature Engineering

Because we are going to be thinking of this in terms of a simple neural network here (like a dense neural network), extend the data by the input data actually being the past $n$ days ($n$ between 1 and 7). In other words the $X$ input should contain a lag of variables you loaded, but lagged by days from 1 through 7. In other words if it hasn't snowed in the past 7 days you will have attributes $[0,0,0,0,0,0,0]$ for yesterday and the preceeding 8 days of no snow, being "columns" or dimensions in your input data.

One challenge is that for weekend you will not have trading days so you will need to do some data filling. After you "fatten" your data, should see if you need all this data. You should normalize all your input variables so that that have an approximate range between 0 and 1. 

In [30]:
#joined_data.info()

In [90]:
joined_data['market_volatility_lag_one'] = pd.Series(0, index=joined_data.index)
joined_data['market_volatility_lag_two'] = pd.Series(0, index=joined_data.index)
joined_data['market_volatility_lag_three'] = pd.Series(0, index=joined_data.index)
joined_data['market_volatility_lag_four'] = pd.Series(0, index=joined_data.index)
joined_data['market_volatility_lag_five'] = pd.Series(0, index=joined_data.index)
joined_data['market_volatility_lag_six'] = pd.Series(0, index=joined_data.index)
joined_data['market_volatility_lag_seven'] = pd.Series(0, index=joined_data.index)
joined_data.head()

,DATE,PRCP,SNOW,Rain_intensity,Snow_intensity,Wind_intensity,Open,High,Low,Close,Adj Close,market_volatility_lag_one,market_volatility_lag_two,market_volatility_lag_three,market_volatility_lag_four,market_volatility_lag_five,market_volatility_lag_six,market_volatility_lag_seven,high-low
0,1992-01-02,0,0,1,0,1,0.465631,0.465631,0.465631,0.465631,0.465631,0,0,0,0,0,0,0,0.0
1,1992-01-03,0,0,1,0,1,0.514483,0.514483,0.514483,0.514483,0.514483,0,0,0,0,0,0,0,0.0
2,1992-01-06,0,0,1,2,1,0.493777,0.493777,0.493777,0.493777,0.493777,0,0,0,0,0,0,0,0.0
3,1992-01-07,1,1,1,2,1,0.487160,0.487160,0.487160,0.487160,0.487160,0,0,0,0,0,0,0,0.0
4,1992-01-08,0,0,0,2,1,0.509512,0.509512,0.509512,0.509512,0.509512,0,0,0,0,0,0,0,0.0


In [91]:
from functools import partial
import datetime

def lag_function(num_days, df,x):
    for i in range(0, len(df)):
        reqDate = df.loc[i, 'DATE'] - datetime.timedelta(days=num_days)
        myIdx = df.index[df['DATE'] == reqDate].tolist()
        if len(myIdx) > 0: 
            df.loc[i,x] = df.loc[myIdx[0], 'SNOW']
        else:
            df.loc[i,x] = 0
    return df

lag_one_day = partial(lag_function, 1)
x = 'market_volatility_lag_one'
joined_data= lag_one_day(joined_data,x)

lag_two_days = partial(lag_function, 2)
x = 'market_volatility_lag_two'
joined_data= lag_two_days(joined_data,x)

lag_three_days = partial(lag_function, 3)
x = 'market_volatility_lag_three'
joined_data= lag_three_days(joined_data,x)

lag_four_days = partial(lag_function, 4)
x = 'market_volatility_lag_four'
joined_data= lag_four_days(joined_data,x)

lag_five_days = partial(lag_function, 5)
x = 'market_volatility_lag_five'
joined_data= lag_five_days(joined_data,x)

lag_six_days = partial(lag_function, 6)
x = 'market_volatility_lag_six'
joined_data= lag_six_days(joined_data,x)

lag_seven_days = partial(lag_function, 7)
x = 'market_volatility_lag_seven'
joined_data= lag_seven_days(joined_data,x)



In [92]:
joined_data.head(10)

,DATE,PRCP,SNOW,Rain_intensity,Snow_intensity,Wind_intensity,Open,High,Low,Close,Adj Close,market_volatility_lag_one,market_volatility_lag_two,market_volatility_lag_three,market_volatility_lag_four,market_volatility_lag_five,market_volatility_lag_six,market_volatility_lag_seven,high-low
0,1992-01-02,0,0,1,0,1,0.465631,0.465631,0.465631,0.465631,0.465631,0,0,0,0,0,0,0,0.0
1,1992-01-03,0,0,1,0,1,0.514483,0.514483,0.514483,0.514483,0.514483,0,0,0,0,0,0,0,0.0
2,1992-01-06,0,0,1,2,1,0.493777,0.493777,0.493777,0.493777,0.493777,0,0,0,0,0,0,0,0.0
3,1992-01-07,1,1,1,2,1,0.487160,0.487160,0.487160,0.487160,0.487160,0,0,0,0,0,0,0,0.0
4,1992-01-08,0,0,0,2,1,0.509512,0.509512,0.509512,0.509512,0.509512,1,0,0,0,0,0,0,0.0
5,1992-01-09,1,1,2,2,1,0.509512,0.509512,0.509512,0.509512,0.509512,0,1,0,0,0,0,0,0.0
6,1992-01-10,1,1,1,2,1,0.456550,0.456550,0.456550,0.456550,0.456550,1,0,1,0,0,0,0,0.0
7,1992-01-13,1,0,1,2,1,0.436664,0.436664,0.436664,0.436664,0.436664,0,0,1,1,0,1,0,0.0
8,1992-01-14,1,1,1,2,0,0.564981,0.564981,0.564981,0.564981,0.564981,0,0,0,1,1,0,1,0.0
9,1992-01-15,1,1,0,2,0,0.576137,0.576137,0.576137,0.576137,0.576137,1,0,0,0,1,1,0,0.0


In [93]:
final_df['ELEMENT'].count()
joined_data.columns

Index(['DATE', 'PRCP', 'SNOW', 'Rain_intensity', 'Snow_intensity',
       'Wind_intensity', 'Open', 'High', 'Low', 'Close', 'Adj Close',
       'market_volatility_lag_one', 'market_volatility_lag_two',
       'market_volatility_lag_three', 'market_volatility_lag_four',
       'market_volatility_lag_five', 'market_volatility_lag_six',
       'market_volatility_lag_seven', 'high-low'],
      dtype='object')

In [94]:
# add the code for dealing with weekends here
joined_data.shape

(253, 19)

## Part 5: Try out different Models and prediction!

Your goal is to predict the volatility in the market, that is the Market High - Market Low your "Y" value. For convenience create that column. All of the other columns will help create your "X" variables. You can use any of the other variables as predictors. Be careful not use Market High or Market Low as "X" variables!

Since we are doing a regression problem that means that the last neural net activation will probably be linear and the loss should be Mean Squared Error or root mean squared error or mean absolute error.

Try five different models. For each model, please report mse, root mse and mean absolute error.  You can get the training history with:

Record the history with:

`history = model.fit(X, y, validation_split=0.1)`

and get the history for your metrics with:

`history.history`

For more details, see this tutorial: https://machinelearningmastery.com/custom-metrics-deep-learning-keras-python/

Also, please note the above tutorial shows you how to include multiple metrics with keras.

Then try cross validation with the above metrics. 

If you've never done cross validation with keras before, please use: https://machinelearningmastery.com/evaluate-performance-deep-learning-models-keras/

The above tutorial will show you how.

After running cross validation for each of the metrics you should be able to answer the following questions:

Is there overfitting? How do you know?
Why do you think certain models worked well and others not as well? 
How might you improve the model?

In [95]:
#joined_data['sub'] = joined_data['High'] - joined_data['Low']
features =  ['PRCP', 'SNOW', 'Rain_intensity', 'Snow_intensity',
       'Wind_intensity','Adj Close',
       'market_volatility_lag_one', 'market_volatility_lag_two',
       'market_volatility_lag_three', 'market_volatility_lag_four',
       'market_volatility_lag_five', 'market_volatility_lag_six',
       'market_volatility_lag_seven']

In [96]:
joined_data['high-low'] = joined_data.High - joined_data.Low

In [97]:
# Separating out the features
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Dropout,Flatten
from sklearn.metrics import mean_absolute_error

X = joined_data.loc[:, features].values
# Separating out the target
#loss=tf.keras.metrics.mean_squared_error
y = joined_data.loc[:,['high-low']].values
# Standardizing the features
X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

linear_reg = LinearRegression()
dt_reg = DecisionTreeRegressor(criterion='mse', max_depth=3)    
svr_linear = SVR(kernel='linear', C=100, gamma='auto')
linear_ridge = Ridge()

# Define model
model = Sequential()
model.add(Dense(100, input_dim=13, activation= "relu"))
model.add(Dense(50, activation= "relu"))
model.add(Dense(50, activation= "relu"))
model.add(Dense(1))
model.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])



linear_reg.fit(X_train, y_train)
dt_reg.fit(X_train, y_train)
svr_linear.fit(X_train, y_train)
linear_ridge.fit(X_train, y_train)
model.fit(X_train, y_train, epochs=10)


C:\Users\sonali\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Train on 202 samples
Epoch 1/10
202/202 [==============================] - 1s 5ms/sample - loss: 0.2486 - mean_squared_error: 0.2486
Epoch 2/10
202/202 [==============================] - 0s 113us/sample - loss: 0.0368 - mean_squared_error: 0.0368
Epoch 3/10
202/202 [==============================] - 0s 130us/sample - loss: 0.0243 - mean_squared_error: 0.0243
Epoch 4/10
202/202 [==============================] - 0s 134us/sample - loss: 0.0078 - mean_squared_error: 0.0078
Epoch 5/10
202/202 [==============================] - 0s 127us/sample - loss: 0.0044 - mean_squared_error: 0.0044
Epoch 6/10
202/202 [==============================] - 0s 117us/sample - loss: 0.0035 - mean_squared_error: 0.0035
Epoch 7/10
202/202 [==============================] - 0s 117us/sample - loss: 0.0020 - mean_squared_error: 0.0020
Epoch 8/10
202/202 [==============================] - 0s 119us/sample - loss: 0.0014 - mean_squared_error: 0.0014
Epoch 9/10
202/202 [==============================] - 0s 122us/sample

In [104]:
models =['linear_regession','svr_linear', 'ann_model']
for model_name, model in zip(models,  [linear_reg, svr_linear, model4]):
    y_pred = model.predict(X_test)
    print("{}  mse:{}".format(model_name, mean_squared_error(y_test,y_pred)))
    print("{}  rmse:{}".format(model_name, np.sqrt(mean_squared_error(y_test,y_pred))))
    print("{}  mae:{}".format(model_name, np.sqrt(mean_absolute_error(y_test,y_pred))))

linear_regession  mse:0.0
linear_regession  rmse:0.0
linear_regession  mae:0.0
svr_linear  mse:0.0
svr_linear  rmse:0.0
svr_linear  mae:0.0
ann_model  mse:0.0002694670066959924
ann_model  rmse:0.016415450243474666
ann_model  mae:0.11573107268983944


here mse, rmse and mae for linear regression and svm model is 0 it ran into underfitting problem 
This can happen if either the model is too simple, or x does not explain y. The latter can have different causes like noise, variables that have an influence but were not observed
I will try with differnt models 

### 5 different models 

In [45]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Dropout,Flatten

def model_one(X_train,y_train):
    model_1 = Sequential()
    model_1.add(Dense(100,input_dim=13,activation='relu'))
    model_1.add(Dense(1, activation='softmax'))
    model_1.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
    model_1.fit(X_train,y_train, validation_split= 0.2,epochs=5,batch_size=5)
    return model_1

def model_two(X_train,y_train):
    model_2 = Sequential()
    model_2.add(Dense(64,input_dim=13,activation='relu'))
    model_2.add(Dense(1, activation='softmax'))
    model_2.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
    model_2.fit(X_train,y_train, validation_split= 0.2,epochs=5,batch_size=3)
    return model_2


def model_three(X_train,y_train):
    model_3 = Sequential()
    model_3.add(Dense(64, input_dim=13, activation='relu'))
    model_3.add(Dropout(0.5))   
    model_3.add(Dense(1, activation='relu'))
    model_3.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
    model_3.fit(X_train,y_train,validation_split= 0.4, epochs=7, batch_size=32)
    return model_3

def model_four(X_train,y_train):
    model_4 = Sequential()
    model_4.add(Dense(100, input_dim=13, activation='relu'))
    model_4.add(Dense(32, activation='relu'))
    model_4.add(Dropout(0.5))
    model_4.add(Dense(1, activation='sigmoid'))
    model_4.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
    model_4.fit(X_train,y_train, epochs=8, batch_size=10)
    return model_4

def model_five(X_train,y_train):
    model_5 = Sequential()
    model_5.add(Dense(64, input_dim=13, activation='relu'))
    model_5.add(Dense(32, activation='relu'))
    model_5.add(Dense(16, activation='relu'))
    model_5.add(Dense(1, activation='sigmoid'))
    model_5.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
    model_5.fit(X_train,y_train, epochs=10, batch_size=10)
    return model_5


In [46]:
model1 = model_one(X_train,y_train)
print(model1.evaluate(X_test,y_test))

Train on 161 samples, validate on 41 samples
Epoch 1/5
161/161 [==============================] - 2s 11ms/sample - loss: 1.0000 - mean_squared_error: 1.0000 - val_loss: 1.0000 - val_mean_squared_error: 1.0000
Epoch 2/5
161/161 [==============================] - 0s 1ms/sample - loss: 1.0000 - mean_squared_error: 1.0000 - val_loss: 1.0000 - val_mean_squared_error: 1.0000
Epoch 3/5
161/161 [==============================] - 0s 1ms/sample - loss: 1.0000 - mean_squared_error: 1.0000 - val_loss: 1.0000 - val_mean_squared_error: 1.0000
Epoch 4/5
161/161 [==============================] - 0s 1ms/sample - loss: 1.0000 - mean_squared_error: 1.0000 - val_loss: 1.0000 - val_mean_squared_error: 1.0000
Epoch 5/5
51/1 [==============================================================================================================================================================================================================================================================================================

In [47]:
# model 2
model2 = model_two(X_train,y_train)
print(model2.evaluate(X_test,y_test))

Train on 161 samples, validate on 41 samples
Epoch 1/5
161/161 [==============================] - 1s 8ms/sample - loss: 1.0000 - mean_squared_error: 1.0000 - val_loss: 1.0000 - val_mean_squared_error: 1.0000
Epoch 2/5
161/161 [==============================] - 0s 954us/sample - loss: 1.0000 - mean_squared_error: 1.0000 - val_loss: 1.0000 - val_mean_squared_error: 1.0000
Epoch 3/5
161/161 [==============================] - 0s 880us/sample - loss: 1.0000 - mean_squared_error: 1.0000 - val_loss: 1.0000 - val_mean_squared_error: 1.0000
Epoch 4/5
161/161 [==============================] - 0s 2ms/sample - loss: 1.0000 - mean_squared_error: 1.0000 - val_loss: 1.0000 - val_mean_squared_error: 1.0000
Epoch 5/5
51/1 [===========================================================================================================================================================================================================================================================================================

In [48]:
model3 = model_three(X_train,y_train)
print(model3.evaluate(X_test,y_test))

Train on 121 samples, validate on 81 samples
Epoch 1/7
121/121 [==============================] - 1s 10ms/sample - loss: 0.0736 - mean_squared_error: 0.0736 - val_loss: 0.0839 - val_mean_squared_error: 0.0839
Epoch 2/7
121/121 [==============================] - 0s 401us/sample - loss: 0.1313 - mean_squared_error: 0.1313 - val_loss: 0.0642 - val_mean_squared_error: 0.0642
Epoch 3/7
121/121 [==============================] - 0s 480us/sample - loss: 0.0351 - mean_squared_error: 0.0351 - val_loss: 0.0490 - val_mean_squared_error: 0.0490
Epoch 4/7
121/121 [==============================] - 0s 451us/sample - loss: 0.0455 - mean_squared_error: 0.0455 - val_loss: 0.0371 - val_mean_squared_error: 0.0371
Epoch 5/7
121/121 [==============================] - 0s 410us/sample - loss: 0.0489 - mean_squared_error: 0.0489 - val_loss: 0.0282 - val_mean_squared_error: 0.0282
Epoch 6/7
121/121 [==============================] - 0s 402us/sample - loss: 0.0299 - mean_squared_error: 0.0299 - val_loss: 0.0213

In [49]:
model4 = model_four(X_train,y_train)
print(model4.evaluate(X_test,y_test))

Train on 202 samples
Epoch 1/8
202/202 [==============================] - 1s 6ms/sample - loss: 0.1777 - mean_squared_error: 0.1777
Epoch 2/8
202/202 [==============================] - 0s 197us/sample - loss: 0.0410 - mean_squared_error: 0.0410
Epoch 3/8
202/202 [==============================] - 0s 168us/sample - loss: 0.0148 - mean_squared_error: 0.0148
Epoch 4/8
202/202 [==============================] - 0s 217us/sample - loss: 0.0071 - mean_squared_error: 0.0071
Epoch 5/8
202/202 [==============================] - 0s 226us/sample - loss: 0.0062 - mean_squared_error: 0.0062
Epoch 6/8
202/202 [==============================] - 0s 172us/sample - loss: 0.0032 - mean_squared_error: 0.0032
Epoch 7/8
202/202 [==============================] - 0s 199us/sample - loss: 0.0026 - mean_squared_error: 0.0026
Epoch 8/8
51/1 [==============================================================================================================================================================================

In [50]:
model5 = model_five(X_train,y_train)
print(model5.evaluate(X_test,y_test))

Train on 202 samples
Epoch 1/10
202/202 [==============================] - 1s 6ms/sample - loss: 0.1592 - mean_squared_error: 0.1592
Epoch 2/10
202/202 [==============================] - 0s 201us/sample - loss: 0.0359 - mean_squared_error: 0.0359
Epoch 3/10
202/202 [==============================] - 0s 171us/sample - loss: 0.0053 - mean_squared_error: 0.0053
Epoch 4/10
202/202 [==============================] - 0s 193us/sample - loss: 0.0015 - mean_squared_error: 0.0015
Epoch 5/10
202/202 [==============================] - 0s 182us/sample - loss: 7.3986e-04 - mean_squared_error: 7.3986e-04
Epoch 6/10
202/202 [==============================] - 0s 176us/sample - loss: 4.6136e-04 - mean_squared_error: 4.6136e-04
Epoch 7/10
202/202 [==============================] - 0s 183us/sample - loss: 3.2711e-04 - mean_squared_error: 3.2711e-04
Epoch 8/10
202/202 [==============================] - 0s 174us/sample - loss: 2.4628e-04 - mean_squared_error: 2.4628e-04
Epoch 9/10
202/202 [=================

## Cross Validation


In [58]:
# implement cross validation here
from sklearn.model_selection import KFold

#X_np = np.array(X)
X_np = np.asarray(X).astype(np.float32)
#y_nn = pd.get_dummies(data = y)
#y_np = np.array(y_nn)
y_np = np.asarray(y).astype(np.float32)

kf = KFold(n_splits=5, random_state=None, shuffle=True)
for train_index, test_index in kf.split(X_np):
   # print("TRAIN:", train_index, "TEST:", test_index)
    x_train,x_test=X_np[train_index],X_np[test_index]
    y_train,y_test=y_np[train_index],y_np[test_index]
    #model 1
    print('model 1')
    model_cv1 = model_one(x_train,y_train)
    print(model_cv1.evaluate(x_test,y_test))
    
    # model 2
    print('model 2')
    model_cv2 = model_two(x_train,y_train)
    print(model_cv2.evaluate(x_test,y_test))
    
    #model 3
    print('model 3')
    model_cv3 = model_three(x_train,y_train)
    print(model_cv3.evaluate(x_test,y_test))
    
    #model 4
    print('model 4')
    model_cv4 = model_four(x_train,y_train)
    print(model_cv4.evaluate(x_test,y_test))
    
    #model 5
    print('model 5')
    model_cv5 = model_five(x_train,y_train)
    print(model_cv5.evaluate(x_test,y_test))

model 1
Train on 161 samples, validate on 41 samples
Epoch 1/5
161/161 [==============================] - 1s 7ms/sample - loss: 1.0000 - mean_squared_error: 1.0000 - val_loss: 1.0000 - val_mean_squared_error: 1.0000
Epoch 2/5
161/161 [==============================] - 0s 809us/sample - loss: 1.0000 - mean_squared_error: 1.0000 - val_loss: 1.0000 - val_mean_squared_error: 1.0000
Epoch 3/5
161/161 [==============================] - 0s 803us/sample - loss: 1.0000 - mean_squared_error: 1.0000 - val_loss: 1.0000 - val_mean_squared_error: 1.0000
Epoch 4/5
161/161 [==============================] - 0s 871us/sample - loss: 1.0000 - mean_squared_error: 1.0000 - val_loss: 1.0000 - val_mean_squared_error: 1.0000
Epoch 5/5
51/1 [=================================================================================================================================================================================================================================================================================

202/202 [==============================] - 0s 326us/sample - loss: 0.0062 - mean_squared_error: 0.0062
Epoch 5/8
202/202 [==============================] - 0s 281us/sample - loss: 0.0051 - mean_squared_error: 0.0051
Epoch 6/8
202/202 [==============================] - 0s 279us/sample - loss: 0.0023 - mean_squared_error: 0.0023
Epoch 7/8
202/202 [==============================] - 0s 309us/sample - loss: 0.0026 - mean_squared_error: 0.0026
Epoch 8/8
51/1 [==============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

161/161 [==============================] - 0s 1ms/sample - loss: 1.0000 - mean_squared_error: 1.0000 - val_loss: 1.0000 - val_mean_squared_error: 1.0000
Epoch 5/5
51/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

202/202 [==============================] - 0s 310us/sample - loss: 4.9089e-04 - mean_squared_error: 4.9089e-04
Epoch 9/10
202/202 [==============================] - 0s 374us/sample - loss: 3.8132e-04 - mean_squared_error: 3.8132e-04
Epoch 10/10
51/1 [=============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.08443720142046611, 0.0844372]
model 4
Train on 202 samples
Epoch 1/8
202/202 [==============================] - 1s 5ms/sample - loss: 0.1795 - mean_squared_error: 0.1795
Epoch 2/8
202/202 [==============================] - 0s 177us/sample - loss: 0.0545 - mean_squared_error: 0.0545
Epoch 3/8
202/202 [==============================] - 0s 230us/sample - loss: 0.0190 - mean_squared_error: 0.0190
Epoch 4/8
202/202 [==============================] - 0s 210us/sample - loss: 0.0120 - mean_squared_error: 0.0120
Epoch 5/8
202/202 [==============================] - 0s 213us/sample - loss: 0.0076 - mean_squared_error: 0.0076
Epoch 6/8
202/202 [==============================] - 0s 196us/sample - loss: 0.0061 - mean_squared_error: 0.0061
Epoch 7/8
202/202 [==============================] - 0s 193us/sample - loss: 0.0045 - mean_squared_error: 0.0045
Epoch 8/8
51/1 [=====================================================================================================================================

162/162 [==============================] - 0s 1ms/sample - loss: 1.0000 - mean_squared_error: 1.0000 - val_loss: 1.0000 - val_mean_squared_error: 1.0000
Epoch 3/5
162/162 [==============================] - 0s 1ms/sample - loss: 1.0000 - mean_squared_error: 1.0000 - val_loss: 1.0000 - val_mean_squared_error: 1.0000
Epoch 4/5
162/162 [==============================] - 0s 1ms/sample - loss: 1.0000 - mean_squared_error: 1.0000 - val_loss: 1.0000 - val_mean_squared_error: 1.0000
Epoch 5/5
50/1 [=========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

203/203 [==============================] - 0s 284us/sample - loss: 5.6247e-04 - mean_squared_error: 5.6247e-04
Epoch 6/10
203/203 [==============================] - 0s 306us/sample - loss: 3.9469e-04 - mean_squared_error: 3.9469e-04
Epoch 7/10
203/203 [==============================] - 0s 288us/sample - loss: 2.9635e-04 - mean_squared_error: 2.9635e-04
Epoch 8/10
203/203 [==============================] - 0s 302us/sample - loss: 2.2936e-04 - mean_squared_error: 2.2936e-04
Epoch 9/10
203/203 [==============================] - 0s 243us/sample - loss: 1.8316e-04 - mean_squared_error: 1.8316e-04
Epoch 10/10
50/1 [===============================================================================================================================================================================================================================================================================================================================================================================================

121/121 [==============================] - 0s 496us/sample - loss: 0.1178 - mean_squared_error: 0.1178 - val_loss: 0.0773 - val_mean_squared_error: 0.0773
Epoch 7/7
50/1 [=============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [59]:
X_train.shape
#y_train.shape

(202, 13)

In [65]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
linear_reg.fit(X_train, y_train)
dt_reg.fit(X_train, y_train)
svr_linear.fit(X_train, y_train)
linear_ridge.fit(X_train, y_train)
model.fit(X_train, y_train, epochs=5)

Train on 202 samples
Epoch 1/5
202/202 [==============================] - 0s 275us/sample - loss: 7.6774e-05 - mean_squared_error: 7.6774e-05
Epoch 2/5
202/202 [==============================] - 0s 140us/sample - loss: 6.3207e-05 - mean_squared_error: 6.3207e-05
Epoch 3/5
 32/202 [===>..........................] - ETA: 0s - loss: 4.5895e-05 - mean_squared_error: 4.5895e-05

C:\Users\sonali\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


202/202 [==============================] - 0s 111us/sample - loss: 4.7028e-05 - mean_squared_error: 4.7028e-05
Epoch 4/5
202/202 [==============================] - 0s 114us/sample - loss: 3.5120e-05 - mean_squared_error: 3.5120e-05
Epoch 5/5
202/202 [==============================] - 0s 118us/sample - loss: 3.0673e-05 - mean_squared_error: 3.0673e-05


In [67]:
##model 1
from sklearn.metrics import mean_absolute_error
print("{}  mse:{}".format(model1, mean_squared_error(y_test,y_pred)))
print("{}  rmse:{}".format(model1, np.sqrt(mean_squared_error(y_test,y_pred))))
print("{}  mae:{}".format(model1, np.sqrt(mean_absolute_error(y_test,y_pred))))

<tensorflow.python.keras.engine.sequential.Sequential object at 0x000002EF0BC0ABA8>  mse:0.002580376579494064
<tensorflow.python.keras.engine.sequential.Sequential object at 0x000002EF0BC0ABA8>  rmse:0.05079740721231807
<tensorflow.python.keras.engine.sequential.Sequential object at 0x000002EF0BC0ABA8>  mae:0.17486787747438276


In [116]:
## model2
print("{}  mse:{}".format(model3, mean_squared_error(y_test,y_pred)))
print("{}  rmse:{}".format(model3, np.sqrt(mean_squared_error(y_test,y_pred))))
print("{}  mae:{}".format(model3, np.sqrt(mean_absolute_error(y_test,y_pred))))

<tensorflow.python.keras.engine.sequential.Sequential object at 0x000002EF3E0D1198>  mse:0.0002694670066959924
<tensorflow.python.keras.engine.sequential.Sequential object at 0x000002EF3E0D1198>  rmse:0.016415450243474666
<tensorflow.python.keras.engine.sequential.Sequential object at 0x000002EF3E0D1198>  mae:0.11573107268983944


In [115]:
## model3
print("{}  mse:{}".format(model3, mean_squared_error(y_test,y_pred)))
print("{}  rmse:{}".format(model3, np.sqrt(mean_squared_error(y_test,y_pred))))
print("{}  mae:{}".format(model3, np.sqrt(mean_absolute_error(y_test,y_pred))))

<tensorflow.python.keras.engine.sequential.Sequential object at 0x000002EF3E0D1198>  mse:0.0002694670066959924
<tensorflow.python.keras.engine.sequential.Sequential object at 0x000002EF3E0D1198>  rmse:0.016415450243474666
<tensorflow.python.keras.engine.sequential.Sequential object at 0x000002EF3E0D1198>  mae:0.11573107268983944


In [114]:
## model4
print("{}  mse:{}".format(model4, mean_squared_error(y_test,y_pred)))
print("{}  rmse:{}".format(model4, np.sqrt(mean_squared_error(y_test,y_pred))))
print("{}  mae:{}".format(model4, np.sqrt(mean_absolute_error(y_test,y_pred))))

<tensorflow.python.keras.engine.sequential.Sequential object at 0x000002EF3D9FFA90>  mse:0.0002694670066959924
<tensorflow.python.keras.engine.sequential.Sequential object at 0x000002EF3D9FFA90>  rmse:0.016415450243474666
<tensorflow.python.keras.engine.sequential.Sequential object at 0x000002EF3D9FFA90>  mae:0.11573107268983944


answer answer overfitting here and analysis here.  
After cross validation data each model's mse imporved but is still has problem of underfitting and overfitting

## Overall Conclusion

Conclude with a full report here on what we know now about this problem. How well it does verses baseline, what the best Keras archtecture is, what features should be used, how the data should be cleaned etc.

The code ran into overfitting, it is usefull but it doesnot solved the problem. we can try diffenrt methods to solve these problem like
1.Cross Validation, 2. Early stoping, 3. Regularization, 4. Training on more data, 5. Remove features

* Reduce your model complexity, which often means reducing the number of parameters.
* Regularization, which basically comes down to constraints on the parameters.
* Use more training data

To improve model performance we can increase the dataset and use Regularizaion